# LCEL (대화내용 기억하기): 메모리 추가

임의의 체인에 메모리를 추가하는 방법을 보여줍니다. 현재 메모리 클래스를 사용할 수 있지만 수동으로 연결해야 합니다


In [21]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [22]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
# model = ChatOpenAI()
from langserve import RemoteRunnable

model = RemoteRunnable(f"{os.getenv('LANGSERVE_ENDPOINT')}/llm-ko")

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

대화내용을 저장할 메모리인 `ConversationBufferMemory` 생성하고 `return_messages` 매개변수를 `True`로 설정하여, 생성된 인스턴스가 메시지를 반환하도록 합니다.

- `memory_key` 설정: 추후 Chain 의 `prompt` 안에 대입될 key 입니다. 변경하여 사용할 수 있습니다.


In [23]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

저장된 대화기록을 확인합니다. 아직 저장하지 않았으므로, 대화기록은 비어 있습니다.


In [24]:
memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

{'chat_history': []}

`RunnablePassthrough.assign`을 사용하여 `chat_history` 변수에 `memory.load_memory_variables` 함수의 결과를 할당하고, 이 결과에서 `chat_history` 키에 해당하는 값을 추출합니다.


In [25]:
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
)

In [26]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [27]:
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

`runnable` 에 첫 번째 대화를 시작합니다.

- `input`: 사용자 입력 대화가 전달됩니다.
- `chat_history`: 대화 기록이 전달됩니다.


In [29]:
runnable.invoke({"input": "hi!"})

{'input': 'hi!', 'chat_history': []}

In [30]:
chain = runnable | prompt | model

첫 번째 대화를 진행합니다.


In [31]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 테디입니다."})
print(response.content)  # 생성된 응답을 출력합니다.

안녕하세요, 테디! 저는 당신을 돕기 위해 고생하는 AI 친구에요. Nameobby를 만난 걸 환영합니다. 앞으로 즐거운 대화와 유익한 정보를 함께 공유할 수 있길 바랍니다. 언제든지 궁금한 점이나 이야기할 것이 있으면 말씀해 주세요!


In [32]:
memory.load_memory_variables({})

{'chat_history': []}

`memory.save_context` 함수는 입력 데이터(`inputs`)와 응답 내용(`response.content`)을 메모리에 저장하는 역할을 합니다. 이는 AI 모델의 학습 과정에서 현재 상태를 기록하거나, 사용자의 요청과 시스템의 응답을 추적하는 데 사용될 수 있습니다.


In [33]:
# 입력된 데이터와 응답 내용을 메모리에 저장합니다.
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 테디입니다."}, {"ai": response.content}
)

# 저장된 대화기록을 출력합니다.
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 테디입니다.'),
  AIMessage(content='안녕하세요, 테디! 저는 당신을 돕기 위해 고생하는 AI 친구에요. Nameobby를 만난 걸 환영합니다. 앞으로 즐거운 대화와 유익한 정보를 함께 공유할 수 있길 바랍니다. 언제든지 궁금한 점이나 이야기할 것이 있으면 말씀해 주세요!')]}

이름을 기억하고 있는지 추가 질의합니다.


In [ ]:
# 이름을 기억하고 있는지 추가 질의합니다.
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
# 답변을 출력합니다.
print(response.content)

## 커스텀 ConversationChain 구현 예시

In [ ]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# ChatOpenAI 모델을 초기화합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [ ]:
class MyConversationChain(Runnable):

    def __init__(self, llm, prompt, memory, input_key="input"):

        self.prompt = prompt
        self.memory = memory
        self.input_key = input_key

        self.chain = (
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables)
                | itemgetter(memory.memory_key)  # memory_key 와 동일하게 입력합니다.
            )
            | prompt
            | llm
            | StrOutputParser()
        )

    def invoke(self, query, configs=None, **kwargs):
        answer = self.chain.invoke({self.input_key: query})
        self.memory.save_context(inputs={"human": query}, outputs={"ai": answer})
        return answer

In [ ]:
# ChatOpenAI 모델을 초기화합니다.
# llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
from langserve import RemoteRunnable

llm = RemoteRunnable(f"{os.getenv('LANGSERVE_ENDPOINT')}/llm-ko")

# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

# 요약 메모리로 교체할 경우
# memory = ConversationSummaryMemory(
#     llm=llm, return_messages=True, memory_key="chat_history"
# )

conversation_chain = MyConversationChain(llm, prompt, memory)

In [ ]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.")

In [ ]:
conversation_chain.invoke("제 이름이 뭐라고요?")

In [ ]:
conversation_chain.invoke("앞으로는 영어로만 답변해주세요 알겠어요?")

In [ ]:
conversation_chain.invoke("제 이름을 다시 한 번 말해주세요")

In [ ]:
conversation_chain.memory.load_memory_variables({})["chat_history"]